# Bayesian AGN Decomposition Analysis for SDSS Spectra (Python 2, version 6.1.2) 

###  Remington O. Sexton (UCR), William Matzko (GMU), Nicholas Darden (UCR), 

some would say its pretty BADASS...

#### Version History: 

##### Versions 1-5
- Very unstable, lots of bugs, kinda messy, not many options or features.  We've made a lot of front- and back-end changes and improvements.  Versions 1-4 were not very flexible, and were originally optimized for Keck LRIS spectra (See [Sexton et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...878..101S/abstract)) and then optimized for large samples of SDSS spectra.  In Version 5 we performed a complete overhaul with more options, features.  The most improved-upon feature was the addition of autocorrelation analysis for paramter chain convergence, which now produces the most robust estimates.

##### Version 6.0.1
- Improved autocorrelation analysis and options.  One can now choose the number of autocorrelation times and tolerance for convergence.  Posterior sampling now restarts if solution jumps prematurely out of convergence.
- Simplified the Jupyter Notebook control panel and layout.  Most of the BADASS machinery is now contained in the badass_v6_0.py file.
- Output of black hole mass based on availability of broad line (based on [Woo et al. (2015)](https://ui.adsabs.harvard.edu/abs/2015ApJ...801...38W/abstract) H$\alpha$ BH mass estimate, and [Sexton et al. (2019)](https://ui.adsabs.harvard.edu/abs/2019ApJ...878..101S/abstract) H$\beta$ BH mass estimate.
- Output of systemic stellar velocity (redshift) and it's uncertainty.
- Output of BPT diagnostic ratios and plot if both H$\alpha$ and H$\beta$ regions are fit simultaneously.
- Minor memory leak improvements by optimizing plotting functions and deleting large arrays from memory via garbage collection.  This is still under development.
- Fixed issues with the outflow test function

##### Version 6.0.2
- Bug fixes

##### Version 6.0.3
- Added minimum S/N option for fitting the LOSVD

##### Version 6.1.0
- MCMC fitting with emcee is now optional with `mcmc_fit`; one can fit using only Monte Carlo bootstrapping with any number of `max_like_niter` iterations to estimate uncertainties if one does not require a fit of the LOSVD.  If you need LOSVD measurements, you still must (and *should*) use emcee.
- One can now perform more than a single maximum likelihood fit for intial parameter values for emcee by changing `max_like_niter`, be advised this will take longer for large regions of spectra, but generally produces better initial parameter values.
- BPT diagnostic classification includes the classic Kewley+01 & Kauffmann+03 diagram to separate starforming from AGN dominated objects, but also the [SII] diagnostic to distinguish Seyferts from LINERs.  The BPT classification is now written to the log file. 
- Cleaning up code + bug fixes.

##### Version 6.1.1
- Store autocorrelation times and tolerances for each paramter in a dictionary and save to a `.npy` file
- More cleaning up Notebook
##### Version 6.1.2
- Bug fixes

In [ ]:
import glob
import time
import natsort
from IPython.display import clear_output

# To see full list of imported packages and modules, see 
# badass_6_1_1.py file.
import badass_v6_1_2 as badass # <<<--- Import BADASS here

### BADASS Options

In [ ]:
################################### Options ################################################################
# Fitting parameters
fit_reg       = (4400,5800) # Fitting region; MILES Stellar Library=(3540,7409); Indo-US Library=(3460,9464)
good_thresh   = 0.50 # good pixels threshold; if number of good pixels is below this percentage, do NOT fit the spectrum.
# Outflow testing parameters
#    BADASS tests for outflows using the region from 4400 A -5800 A to constrain the effects of FeII when fitting for outflows.
#    If 'test_outflows=True', then BADASS fits ALL fitting components in the region, and performs 'outflow_test_niter' monte-carlo
#    bootstrap simulations by resampling the residual noise of the original fit.  If 'test_outflows=False', then it will skip this
#    force whatever fitting components you choose.
test_outflows      = True 
outflow_test_niter = 10 # number of monte carlo bootstrap simulations for outflows
# Maximum Likelihood parameters
max_like_niter = 1 # number of maximum likelihood parameters
# LOSVD parameters
min_sn_losvd  = 2  # minimum S/N threshold for fitting the LOSVD; below this limit a host template is used.
# MCMC algorithm parameters #################################################################################
# Autocorrelation & MCMC Parameters
mcmc_fit      = True # Perform robust fitting using emcee
nwalkers      = 100  # Number of emcee walkers; minimum = 2 x ( # number of parameters)
auto_stop     = True # Automatic stop using autocorrelation analysis
conv_type     = 'median' # 'median','mean', or list of specific paramters, eg. ('stel_vel','stel_disp')
min_samp      = 2500  # minimum number of iterations for sampling post-convergence; recommended minimum min_samp=1000
ncor_times    = 10    # number of autocorrelation times for convergence; recommended minimum ncor_times=2
autocorr_tol  = 10.0   # percentile tolderance between checking autocorrelation times; recommended minimum autocorr_tol = 5.0 
write_iter    = 100   # check every N number of iterations
write_thresh  = 100   # when to start writing parameters to MCMC_chain.csv
burn_in       = 47500 # (only works if auto_stop = False)
min_iter      = 100   # minimum number of iterations before checking for convergence
max_iter      = 50000 # max number of MCMC iterations before stopping
#############################################################################################################
# Final component fitting options
    # Note: by default, all options (except for fit_losvd) are set to 'True' for the initial fit, so 
    # as to make no assumptions on the presence of outflows
fit_feii      = True #True # fit FeII (option for Type 2 AGN)
fit_losvd     = True # fit LOSVD (Stellar population) in final model only
fit_host      = True # fit host-galaxy using a template(used when LOSVD is false or total continuum S/N < 5)
fit_power     = True #True # fit AGN power-law continuum (option for Type 2 AGN)
fit_broad     = True #True # fit broad lines (option for Type 2 AGN)
fit_narrow    = True # fit narrow lines (because why not)
fit_outflows  = True # fit outflows;
tie_narrow    = False # tie all narrow components to [OIII]5007 (also ties outflow components); Note: Only works if wavelength
                      # range includes the Hb/[OIII] region.
#############################################################################################################
# Multiprocessing options
threads = 4 # number of processes per object

### Run BADASS

The following is shows the suggested directory structure for fitting multiple (or even single) SDSS spectra.  It also shows the required calling sequence and necessary parameters for BADASS.

In [ ]:
### Directory Structure ##########################################################
spec_dir = 'example_spectra/' # Directory where spectral folders are kept
ppxf_dir = 'badass_data_files/' # Directory where BADASS support files 
temp_dir = ppxf_dir+'indo_us_library' # Directory where stellar templates are kept
# Get full list of spectrum folders; these will be the working directories
spec_loc = natsort.natsorted( glob.glob(spec_dir+'*') )
# ###########################################################################################################

# Loop over all objects we want to fit in spec_loc
for i in range(0,1,1): # run for first object in spec_loc  
    clear_output() # Clears notebook output after each object

    work_dir = spec_loc[i]+'/' # working directory; where MCMC_output_# folders for each run are kept
    run_dir,prev_dir = badass.setup_dirs(work_dir) # This function sets up working directories for BADASS
    file = glob.glob(work_dir+'*.fits')[0] # Get name of FITS spectra file
    #
    print('\n Starting object %s...' % (spec_loc[i][-19:]) )
    start_time = time.time() # start timer

    # Call the main function in BADASS
    badass.run_BADASS(file,run_dir,temp_dir,
                      fit_reg=fit_reg, good_thresh=good_thresh,
                      test_outflows=test_outflows, outflow_test_niter=outflow_test_niter,
                      max_like_niter=max_like_niter, min_sn_losvd=min_sn_losvd,
                      mcmc_fit=mcmc_fit, nwalkers=nwalkers, auto_stop=auto_stop, conv_type=conv_type, 
                      min_samp=min_samp, ncor_times=ncor_times, autocorr_tol=autocorr_tol,
                      write_iter=write_iter, write_thresh=write_thresh, burn_in=burn_in,
                      min_iter=min_iter, max_iter=max_iter,
                      fit_feii=fit_feii, fit_losvd=fit_losvd, fit_host=fit_host, fit_power=fit_power,
                      fit_broad=fit_broad, fit_narrow=fit_narrow, fit_outflows=fit_outflows, tie_narrow=tie_narrow,
                      threads=threads)
    
    elap_time = (time.time() - start_time)
    print("\n Total Runtime = %s" % (badass.time_convert(elap_time)))
    print('\n Done! \n')